# Scraping Researcher Ages on Veromi

## Python Setup

In [1]:
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import unidecode
from veromi_query import query

## List of Individuals

Let's load in the list of individuals for whom we want to scrape the age. This list already has a unique ID variable per inventor.

In [88]:
ind = pd.read_csv('../data/raw/ind_info.csv', low_memory=False)

In [89]:
del ind['Unnamed: 0'], ind['selected_subset'], ind['last_name']
ind.rename(columns={'full_last_name': 'last_name', 'inventor_id':'id'}, inplace = True)

In [90]:
ind.describe(include='all')

,id,first_name,middle_name,last_name,city,state,country
count,4980929,4979455,2223368,4979597,4970991,2264690,4975800
unique,3663409,194144,49122,631580,103621,61,249
top,4090850-1,John,J.,Lee,Tokyo,CA,US
freq,200,94614,169824,37219,124683,447839,2266802


In [91]:
ind = ind[ind['country']=="US"].reset_index(drop=True)

In [92]:
ind.describe(include='all')

,id,first_name,middle_name,last_name,city,state,country
count,2266802,2266745,1637819,2266783,2265349,2264690,2266802
unique,1796830,86199,28147,357823,22141,61,1
top,5766379-2,John,J.,Smith,San Jose,CA,US
freq,67,78303,142608,12895,33910,447839,2266802


### Data Cleanup

In [93]:
string_vars = ['first_name', 'middle_name', 'last_name', 'city', 'state', 'country']
for var in string_vars:
    ind[var] = ind[var].str.upper().str.strip().fillna('')

__TO DO: Remove "Jr."__

### Export to CSV

In [10]:
ind.to_csv('../data/ind_clean.csv', index=False)

In [11]:
ind.head(10000).to_csv('../data/ind_clean_sample.csv', index=False)

## Scrape Profiles

__Run `age_scraper.py` code.__

The following code works for a limited number of queries.

```
import pandas as pd
from veromi_query import query

# Load in data:
ind = pd.read_csv('../data/ind_clean.csv', low_memory=False)
ind = ind.fillna('')

# Query Veromi:
profiles = query([[0, ind, ind.shape[0]])

# Export to CSV:
profiles.to_csv('../data/profiles_ind_clean.csv', index=False)
```

When running a large number of queries, I adapt the code for parallelization. Here is the example for 3 parallel tasks. In reality, I used a clustered computing system and 100 parallel tasks.

```
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from veromi_query import query


# Load in data:
ind = pd.read_csv('../data/ind_clean.csv', low_memory=False)
ind = ind.fillna('')

# Define Pool:
pool = Pool(processes=3)

# Run query for the job:
profiles0 = pool.apply_async(query, [[0, ind, 1000000]])
profiles1 = pool.apply_async(query, [[1, ind, 1000000]])
profiles2 = pool.apply_async(query, [[2, ind, 1000000]])
profiles0 = profiles0.get()
profiles1 = profiles1.get()
profiles2 = profiles2.get()

# Combine Frames:
frames = [profiles0, profiles1, profiles2]

profiles = pd.concat(frames).reset_index(drop=True)

# Export results as CSV
profiles.to_csv('../data/.csv', index=False)
```

## Parse Age from Data

In [2]:
nb_processes = 100

In [3]:
# Load in entire dataset
profiles = pd.read_csv('../data/profiles_ind_clean.csv')

In [4]:
profiles.head()

,id,name
0,0.0,"SYFRITT, HAROLD ARTHUR (Age 79)"
1,1.0,"SYFRITT, HAROLD ARTHUR (Age 79)"
2,2.0,"BAILIN, RICHARD C (Age 91)"
3,2.0,"BAILIN, RICHARD"
4,3.0,"VANBRAKEL, RUSSEL A (Age 81)"


In [5]:
profiles['age'] = profiles['name'].str.extract(r'(\(Age\s\d+\))', expand=True)
profiles['age'] = pd.to_numeric(profiles['age'].str.extract(r'(\d+)', expand=False))

In [58]:
temp = profiles.sort_values(['id', 'name'])

In [94]:
temp.head()

,id,name,age
0,0.0,"SYFRITT, HAROLD ARTHUR (Age 79)",79.0
1,1.0,"SYFRITT, HAROLD ARTHUR (Age 79)",79.0
3,2.0,"BAILIN, RICHARD",NaN
2,2.0,"BAILIN, RICHARD C (Age 91)",91.0
5,3.0,"VAN BRAKEL, BRAKEL RUSSELL",NaN


For now, let's keep the most common age associated to a given Individual\*Location ID. This also gets rid on `NaN` values.

In [17]:
age_lookup = profiles.groupby(['id', 'age']).size().reset_index().rename(columns={0: 'match_count'})
age_lookup = age_lookup.sort_values(['id', 'match_count'], ascending=[True, False])
age_lookup = age_lookup.drop_duplicates('id', keep='first')

Let's also collect some matching stats.

In [14]:
matching_stats = profiles.groupby(['id'])
matching_stats = pd.DataFrame({'nb_matches': matching_stats.size(),
                               'nb_matches_with_age': matching_stats['age'].count(),
                               'nb_unique_ages': matching_stats['age'].nunique()
                              }).reset_index()

## Merge back onto the Original Data of Individuals\*Locations

In [96]:
ind = pd.read_csv('../data/ind_clean.csv')

In [97]:
ind = pd.merge(ind, age_lookup, how='left', on='id')
ind = pd.merge(ind, matching_stats, how='left', on='id')
ind['nb_matches'].fillna(0, inplace=True)
ind['nb_matches_with_names'].fillna(0, inplace=True)
ind['nb_unique_ages'].fillna(0, inplace=True)

In [98]:
ind.describe(include='all')

,inventor_id,first_name,middle_name,last_name,city,state,country,id,age,match_count,nb_matches,nb_matches_with_names,nb_unique_ages
count,2266802,2266711,1637791,2266688,2265349,2264690,2266802,2266802.0,1.876797e+06,1.876797e+06,2.266802e+06,2.266802e+06,2.266802e+06
unique,1796830,85489,27940,355369,22125,61,1,2266802.0,NaN,NaN,NaN,NaN,NaN
top,5766379-2,JOHN,J.,SMITH,SAN JOSE,CA,US,2266801.0,NaN,NaN,NaN,NaN,NaN
freq,67,78303,142609,12895,33910,447839,2266802,1.0,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.974880e+01,1.299313e+00,4.076881e+00,2.675753e+00,2.181955e+00
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510506e+01,7.314095e-01,8.951167e+00,5.432650e+00,3.918751e+00
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.900000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.800000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.900000e+01,1.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00


In [101]:
ind['nb_unique_ages_t'] = np.where(ind['nb_unique_ages']<5, ind['nb_unique_ages'].apply(lambda x: str(x)), ">=5")
print("Number of unique ages by inventor:")
print(ind['nb_unique_ages_t'].value_counts(normalize=True))
del ind['nb_unique_ages_t']

Number of unique ages by inventor:
1.0    0.568640
0.0    0.172051
>=5    0.101502
2.0    0.100101
3.0    0.036750
4.0    0.020956
Name: nb_unique_ages_t, dtype: float64


### Assign Unique Age to Individual ID

In [83]:
ind_id_age = ind.groupby(['inventor_id', 'age'])['match_count'].sum().reset_index()
ind_id_age = ind_id_age.sort_values(['inventor_id', 'match_count'], ascending=[True, False])
ind_id_age = ind_id_age.drop_duplicates(['inventor_id'], keep='first')

# Merge to unique list of Individual IDs
unique_inds = ind[['inventor_id', 'first_name', 'middle_name', 'last_name']].drop_duplicates()
ind_id_age = pd.merge(unique_inds, ind_id_age, how='left', on='inventor_id')

In [84]:
ind_id_age.head()

,inventor_id,first_name,middle_name,last_name,age,match_count
0,3930274-1,HAROLD,A.,SYFRITT,79.0,2.0
1,3930275-1,RICHARD,NaN,BAILIN,91.0,1.0
2,3930276-1,RUSSEL,A.,VAN BRAKEL,81.0,2.0
3,3930277-1,RICHARD,F.,WULFF,80.0,5.0
4,3930278-1,RICHARD,A.,NASCA,87.0,3.0


In [85]:
ind_id_age.describe(include='all')

,inventor_id,first_name,middle_name,last_name,age,match_count
count,1796830,1796754,1278672,1796733,1.494904e+06,1.494904e+06
unique,1796830,85489,27940,355369,NaN,NaN
top,8754633-2,JOHN,J.,SMITH,NaN,NaN
freq,1,59092,110732,8210,NaN,NaN
mean,NaN,NaN,NaN,NaN,6.002239e+01,1.497659e+00
std,NaN,NaN,NaN,NaN,1.535876e+01,1.015337e+00
min,NaN,NaN,NaN,NaN,1.800000e+01,1.000000e+00
25%,NaN,NaN,NaN,NaN,4.900000e+01,1.000000e+00
50%,NaN,NaN,NaN,NaN,5.800000e+01,1.000000e+00
75%,NaN,NaN,NaN,NaN,6.900000e+01,2.000000e+00


In [87]:
ind_id_age.to_csv('../output/ind_age.csv', index=False)

## Sandbox